In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout, Input, Layer, Lambda, GlobalAveragePooling1D
)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.datasets import imdb

# Load IMDB dataset
num_words = 10000
maxlen = 200
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

# Pad sequences to the same length
x_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(X_test, maxlen=maxlen)

# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Define an Attention Layer
class Attention(Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def call(self, inputs):
        query, value = inputs, inputs
        scores = tf.matmul(query, value, transpose_b=True)  # Shape: (batch_size, seq_length, seq_length)
        distribution = tf.nn.softmax(scores, axis=-1)  # Attention weights
        context = tf.matmul(distribution, value)  # Shape: (batch_size, seq_length, embedding_dim)
        return context

# Build the Bidirectional LSTM model with Attention
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(input_dim=num_words, output_dim=128, input_length=maxlen)(input_layer)
bilstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention_layer = Attention()(bilstm_layer)  # Output: (batch_size, seq_length, embedding_dim)

# Average the attention output across the sequence dimension
pooled_attention = GlobalAveragePooling1D()(attention_layer)

# Add Dropout and Dense layers for binary classification
dropout_layer = Dropout(0.5)(pooled_attention)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

# Build and compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Function to decode the integer-encoded reviews back to words
def decode_review(encoded_review):
    word_index = imdb.get_word_index()
    reverse_word_index = {value: key for key, value in word_index.items()}
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review if i > 2])  # Offset by 3 for padding
    return decoded_review

# Function to predict sentiment for a given review
def predict_sentiment(review):
    word_index = imdb.get_word_index()
    encoded_review = [word_index[word] + 3 for word in review.split() if word in word_index]
    padded_review = pad_sequences([encoded_review], maxlen=maxlen)
    prediction = model.predict(padded_review, verbose=0)  # Disable verbose output
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    return sentiment

# Output sample sentiments from the test set
print("\nSample Sentiments:")
for i in range(5):  # Print 5 sample sentiments
    review = decode_review(X_test[i])
    sentiment = predict_sentiment(review)
    print("-" * 50)
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}")
    print("-" * 50)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 200, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 200, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None, 200, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 210s 658ms/step - accuracy: 0.6773 - loss: 0.5583 - val_accuracy: 0.8736 - val_loss: 0.3077
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 259s 649ms/step - accuracy: 0.9144 - loss: 0.2328 - val_accuracy: 0.8714 - val_loss: 0.3180
782/782 ━━━━━━━━━━━━━━━━━━━━ 71s 91ms/step - accuracy: 0.8627 - loss: 0.3408
Test Accuracy: 86.26%

Sample Sentiments:
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
--------------------------------------------------
Review: please give this one a miss br br and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite so all you madison fans give this a miss
Predicted Sentiment: Negative
--------------------------------------------------
--------------------------------------------------
Review: this film requires a lot of patience because it 